In [1]:
import os
import json
from data_loader.SingleDataLoader import SingleDataLoader
from data_loader.DoubleDataLoader import DoubleDataLoader
from models.LeNet import LeNet
from models.AlexNet import Alexnet
from trainer.train import train_func
from lib.path import get_training_data_dir
from custom_losses.dice import dice_loss, dice_coefficient
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.losses import BinaryFocalCrossentropy, BinaryCrossentropy

2024-04-29 12:37:50.587277: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 12:37:50.587667: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 12:37:50.590632: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 12:37:50.627478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 12:37:51.222099: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
data_dir = '../data'
train_list = os.path.join(data_dir, 'train_list')
test_list = os.path.join(data_dir, 'test_list')
val_list = os.path.join(data_dir, 'val_list')

In [3]:
DATA_TYPE1 = 'gr'
DATA_TYPE2 = 'gist'
DATA_VOXEL_NUM = 10
CLASSIFYING_RULE = 'WaterClassifyingRuleSurface'
LIGAND_POCKET_DEFINER = 'LigandPocketDefinerOriginal'
LIGAND_VOXEL_NUM = 8

training_data_dir1 = get_training_data_dir(DATA_TYPE1, DATA_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER, LIGAND_VOXEL_NUM)
training_data_dir2 = get_training_data_dir(DATA_TYPE2, DATA_VOXEL_NUM, CLASSIFYING_RULE, LIGAND_POCKET_DEFINER, LIGAND_VOXEL_NUM)

print(training_data_dir1, "\n", training_data_dir2)

/home/ito/research/data/training_data/gr/data_voxel_num_10/LigandPocketDefinerOriginal/ligand_pocket_voxel_num_8/WaterClassifyingRuleSurface/ 
 /home/ito/research/data/training_data/gist/data_voxel_num_10/LigandPocketDefinerOriginal/ligand_pocket_voxel_num_8/WaterClassifyingRuleSurface/


In [4]:
data_loader = SingleDataLoader(training_data_dir1)

In [5]:
# data_loader = DoubleDataLoader(training_data_dir1, training_data_dir2)

In [6]:
train_data, train_labels = data_loader.load_data(train_list)
test_data, test_labels = data_loader.load_data(test_list)
val_data, val_labels = data_loader.load_data(val_list)

Error processing 1nhu: No data found for 1nhu
Error processing 1np0: No data found for 1np0
Error processing 1olu: No data found for 1olu
Error processing 1pb8: This is empty data: 1pb8
Error processing 1pb9: This is empty data: 1pb9
Error processing 1y1z: This is empty data: 1y1z
Error processing 1yq7: No data found for 1yq7
Error processing 2d3u: No data found for 2d3u
Error processing 2d3z: No data found for 2d3z
Error processing 2epn: No data found for 2epn
Error processing 2j47: No data found for 2j47
Error processing 2oax: This is empty data: 2oax
Error processing 2p15: This is empty data: 2p15
Error processing 2pog: This is empty data: 2pog
Error processing 2q8h: No data found for 2q8h
Error processing 2vt3: No data found for 2vt3
Error processing 2vvs: No data found for 2vvs
Error processing 2xm2: No data found for 2xm2
Error processing 2ypo: This is empty data: 2ypo
Error processing 2yxj: No data found for 2yxj
Error processing 2zkj: No data found for 2zkj
Error processing 3f7

In [7]:
print('Train data shape: ', train_data.shape)
print('Train labels shape: ', train_labels.shape)
print('Test data shape: ', test_data.shape)
print('Test labels shape: ', test_labels.shape)
print('Val data shape: ', val_data.shape)
print('Val labels shape: ', val_labels.shape)

Train data shape:  (50275, 21, 21, 21, 1)
Train labels shape:  (50275,)
Test data shape:  (13510, 21, 21, 21, 1)
Test labels shape:  (13510,)
Val data shape:  (10317, 21, 21, 21, 1)
Val labels shape:  (10317,)


In [11]:
input_shape = (DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, DATA_VOXEL_NUM*2+1, 1)
epochs = 5
batch_size = 128
n_base = 16
learning_rate = 1e-4
early_stopping = 300
BN = True
dropout = 0.5
model_func = LeNet
losses = [BinaryCrossentropy(), dice_loss]
loss= losses[0]
metrics = ['accuracy', dice_coefficient, Recall(), Precision()]
path_type = '/'.join(training_data_dir1.split('/')[6:11])
checkpoint_path = f"./checkpoints/{path_type}/LeNet/" + "cp-{epoch:04d}.weights.h5"
model_checkpoint = True

In [12]:
pos = train_labels.sum()
neg = train_labels.shape[0] - pos
total = train_labels.shape[0]

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

{0: 1.0425306901128069, 1: 0.9608034246837136}


In [13]:
clf, clf_hist, clf_eval = train_func(
                                    x_train=train_data,
                                    y_train=train_labels,
                                    x_test=test_data,
                                    y_test=test_labels,
                                    x_val=val_data,
                                    y_val=val_labels,
                                    input_shape=input_shape,
                                    model_func=model_func,
                                    loss=loss,
                                    metrics=metrics,
                                    epochs=epochs,
                                    batch_size=batch_size,
                                    n_base=n_base,
                                    learning_rate=learning_rate,
                                    early_stopping=early_stopping,
                                    checkpoint_path=checkpoint_path,
                                    model_checkpoint=model_checkpoint,
                                    class_weight=class_weight,
                                    BN = BN,
                                    dropout=dropout
                                )

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_2 (Conv3D)               │ (None, 21, 21, 21, 16) │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 21, 21, 21, 16) │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 10, 10, 10, 16) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 10, 10, 10, 32) │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 10, 10, 10, 32) │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 5, 5, 5, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │       128,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,561 (556.88 KB)

 Trainable params: 142,465 (556.50 KB)

 Non-trainable params: 96 (384.00 B)

Epoch 1/5
393/393 ━━━━━━━━━━━━━━━━━━━━ 24s 58ms/step - accuracy: 0.5526 - dice_coefficient: 0.5329 - loss: 0.7568 - precision_1: 0.5763 - recall_1: 0.5257 - val_accuracy: 0.6202 - val_dice_coefficient: 0.5283 - val_loss: 0.6526 - val_precision_1: 0.7039 - val_recall_1: 0.4725
Epoch 2/5
393/393 ━━━━━━━━━━━━━━━━━━━━ 22s 57ms/step - accuracy: 0.6446 - dice_coefficient: 0.5684 - loss: 0.6328 - precision_1: 0.6674 - recall_1: 0.6285 - val_accuracy: 0.6625 - val_dice_coefficient: 0.5819 - val_loss: 0.6191 - val_precision_1: 0.6796 - val_recall_1: 0.6710
Epoch 3/5
393/393 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - accuracy: 0.6684 - dice_coefficient: 0.5859 - loss: 0.6104 - precision_1: 0.6966 - recall_1: 0.6395 - val_accuracy: 0.6691 - val_dice_coefficient: 0.5936 - val_loss: 0.6132 - val_precision_1: 0.6884 - val_recall_1: 0.6708
Epoch 4/5
393/393 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - accuracy: 0.6796 - dice_coefficient: 0.5976 - loss: 0.5957 - precision_1: 0.7088 - recall_1: 0.6486 - val_accuracy

In [ ]:
from lib.helper import make_dir

history_save_path = f"./history/{path_type}/LeNet/training_history.json"
make_dir(history_save_path)
with open(history_save_path, 'w') as f:
    json.dump(clf_hist.history, f)

In [ ]:
prediction = clf.predict(test_data)

524/524 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
prediction.round().sum()

2417.0